In [1]:
using PyCall
PyCall.pyversion
@pyimport run_evtol
include("MIGP.jl")

solve_MIGP (generic function with 2 methods)

In [2]:
function branch(cur_val,g,best_val, cleaned_vars)
    (above, below) = nearest_ints(cur_val)

    Above = g(cleaned_vars, [best_val...,above])
    Below = g(cleaned_vars, [best_val...,below])

    (val, cost, I_vr, I_vl) = pick_next((Above...,above), (Below...,below))

    return (val, cost, I_vr, I_vl)
end

branch (generic function with 1 method)

In [3]:
t = @elapsed @time out = solve_MIGP(run_evtol.run_opt)
println(string(out[4])*" calls to solver ("*string(t/out[4])*" sec/call)")

 11.778866 seconds (3.60 M allocations: 176.634 MiB, 0.35% gc time)
25 calls to solver (0.47557837944 sec/call)


In [4]:
@time out = run_evtol.run_opt()
println("Relaxed cost:"*string(out[3])*" lbf")



  0.335879 seconds (4.07 k allocations: 143.641 KiB)
Relaxed cost:638.502 lbf


# Parallel Speedup


In [5]:
using Distributed
addprocs(2)

2-element Array{Int64,1}:
 2
 3

In [6]:
@everywhere using PyCall
@everywhere @pyimport run_evtol


In [7]:
function branch(cur_val,g,best_val, cleaned_vars)
    (above, below) = nearest_ints(cur_val)
    input_list = [(cleaned_vars, [best_val...,above]),(cleaned_vars, [best_val...,below])]
    results = @distributed vcat for inp in input_list
           g(inp...)
       end
    (val, cost, I_vr, I_vl) = pick_next((results[1]...,above), (results[2]...,below))

    return (val, cost, I_vr, I_vl)
end

branch (generic function with 1 method)

In [8]:
t = @elapsed @time out = solve_MIGP(run_evtol.run_opt)
println(string(out[4])*" calls to solver ("*string(t/out[4])*" sec/call)")

  7.804041 seconds (3.27 M allocations: 162.155 MiB, 0.83% gc time)
13 calls to solver (0.6003304403076923 sec/call)


#### Solution checking takes a lot of time 

In [9]:
t = @elapsed @time out = solve_MIGP(run_evtol.run_opt, false)
println(string(out[4])*" calls to solver ("*string(t/out[4])*" sec/call)")

  1.286083 seconds (12.14 k allocations: 527.109 KiB)
4 calls to solver (0.321562484 sec/call)


In [10]:
@everywhere using Dagger
function branch(cur_val,g,best_val, cleaned_vars)
    (above, below) = nearest_ints(cur_val)
    results = Dict()
    input_list = [(cleaned_vars, [best_val...,above]),(cleaned_vars, [best_val...,below])]
    results = collect(delayed(vcat)(delayed(g)(input_list[1]...), delayed(g)(input_list[2]...)))
    #results = @distributed vcat for inp in input_list
    #       g(inp...)
    #   end
    (val, cost, I_vr, I_vl) = pick_next((results[1]...,above), (results[2]...,below))

    return (val, cost, I_vr, I_vl)
end

branch (generic function with 1 method)

In [11]:
@time solve_MIGP(run_evtol.run_opt)

 11.048798 seconds (10.52 M allocations: 522.481 MiB, 1.48% gc time)


(["INT_N_lift_motors_Mission.32", "INT_N_thrust_motors_Mission.32"], [28.0, 2.0], 638.5117938100791, 13)